# CS 109A/AC 209A/STAT 121A Data Science: 2016 Midterm 2 Solutions 
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Tuesday, November 22nd, 2016 at 12:00pm

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Part I: Diagnosing the Semian Flu 2016

You are given the early data for an outbreak of a dangerous virus originating from a group of primates being keeped in a Massechussetts biomedical research lab, this virus is dubbed the "Semian Flu".

You have the medical records of $n$ number of patients in `'flu_train.csv`. There are two general types of patients in the data, flu patients and healthy (this is recorded in the column labeled `flu`, a 0 indicates the absences of the virus and a 1 indicates presence). Furthermore, scientists have found that there are two strains of the virus, each requiring a different type of treatment (this is recorded in the column labeled `flutype`, a 1 indicates the absences of the virus, a 2 indicates presence of strain 1 and a 3 indicates the presence of strain 2).

**Your task:** build a model to predict if a given patient has the flu. Your goal is to catch as many flu patients as possible without misdiagnosing too many healthy patients.

**The deliverable:** a function called `flu_predict` which satisfies:

- input: `x_test`, a set of medical predictors for a group of patients
- output: `y_pred`, a set of labels, one for each patient; 0 for healthy and 1 for infected with the flu virus

The MA state government will use your model to diagnose sets of future patients (held by us). You can expect that there will be an increase in the number of flu patients in any groups of patients in the future.

We provide you with some benchmarks for comparison.

**Baseline Model:** 
- ~50% expected accuracy on healthy patients in observed data
- ~50% expected accuracy on flu patients in observed data
- ~50% expected accuracy on healthy patients in future data 
- ~50% expected accuracy on flu patients in future data
- time to build: 5 min

**Reasonable Model:** 
- ~69% expected accuracy on healthy patients in observed data
- ~55% expected accuracy on flu patients, in observed data
- ~69% expected accuracy on healthy patients in future data
- ~60% expected accuracy on flu patients, in future data
- time to build: 20 min

**Grading:**
Your grade will be based on:
1. your model's ability to out-perform our benchmarks
2. your ability to carefully and thoroughly follow the data science pipeline (see lecture slides for definition)
3. the extend to which all choices are reasonable and defensible by methods you have learned in this class

**Solutions:**

## Step 1: Read the data, clean and explore the data

There are a large number of missing values in the data. Nearly all predictors have some degree of missingness. Not all missingness are alike: as Mike points out, NaN in the `'pregnancy'` column is meaningful and informative, as patients with NaN's in the pregnancy column are males, where as NaN's in other predictors may appear randomly. 


**What we do:** We make no attempt to interpret the predictors and we make no attempt to model the missing values in the data in any meaningful way. We replace all missing values with 0.

However, it would be more complete to look at the data and allow the data to inform your decision on how to address missingness. For columns where NaN values are informative, you might want to treat NaN as a distinct value; You might want to drop predictors with too many missing values and impute the ones with few missing values using KNN or a parametric model. There are many acceptable strategies here, as long as the appropriateness of the method in the context of the task and the data is discussed.

In [2]:
#Train

df = pd.read_csv('data/flu_train.csv')

df = df[~np.isnan(df['flu'])]

df.head()

,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,...,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation,PregnantNow,flu,flutype
0,51624,male,34,30-39,409.0,White,NaN,High School,Married,25000-34999,...,Yes,Yes,16.0,8.0,1.0,No,Heterosexual,NaN,0,1
1,51630,female,49,40-49,596.0,White,NaN,Some College,LivePartner,35000-44999,...,Yes,Yes,12.0,10.0,1.0,Yes,Heterosexual,NaN,0,1
2,51638,male,9,0-9,115.0,White,NaN,NaN,NaN,75000-99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,51646,male,8,0-9,101.0,White,NaN,NaN,NaN,55000-64999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,51647,female,45,40-49,541.0,White,NaN,College Grad,Married,75000-99999,...,No,Yes,13.0,20.0,0.0,Yes,Bisexual,NaN,0,1


In [4]:
#Clean and encode

encode = preprocessing.LabelEncoder()

for column in df.columns:
    if df[column].dtype == np.object:
        df[column] = df[column].fillna('')
        df.loc[:, column] = encode.fit_transform(df[column])
        
df = df.fillna(0)

df.head()

In [9]:
#Test

df_test = pd.read_csv('data/flu_test.csv')

df_test = df_test[~np.isnan(df_test['flu'])]

df_test.head()

,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,...,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation,PregnantNow,flu,flutype
0,51625,male,4,0-9,49.0,Other,NaN,NaN,NaN,20000-24999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,51678,male,60,60-69,721.0,White,NaN,High School,Married,15000-19999,...,No,Yes,20.0,1.0,NaN,No,NaN,NaN,0,1
2,51694,male,38,30-39,458.0,White,NaN,Some College,Married,20000-24999,...,No,Yes,23.0,1.0,1.0,No,Heterosexual,NaN,0,1
3,51695,male,8,0-9,104.0,White,NaN,NaN,NaN,65000-74999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,51711,female,59,50-59,718.0,Other,NaN,8th Grade,Widowed,20000-24999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3


In [12]:
#Clean and encode

encode = preprocessing.LabelEncoder()

for column in df_test.columns:
    if df_test[column].dtype == np.object:
        df_test[column] = df_test[column].fillna('')
        df_test.loc[:, column] = encode.fit_transform(df_test[column])
        
df_test = df_test.fillna(0)

df_test.head()

,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,...,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation,PregnantNow,flu,flutype
0,51625,1,4,1,49.0,3,0,0,0,5,...,0,0,0.0,0.0,0.0,0,0,0,0,1
1,51678,1,60,7,721.0,4,0,4,3,4,...,1,2,20.0,1.0,0.0,1,0,0,0,1
2,51694,1,38,4,458.0,4,0,5,3,5,...,1,2,23.0,1.0,1.0,1,2,0,0,1
3,51695,1,8,1,104.0,4,0,0,0,10,...,0,0,0.0,0.0,0.0,0,0,0,0,1
4,51711,0,59,6,718.0,3,0,1,6,5,...,0,0,0.0,0.0,0.0,0,0,0,1,3


In [14]:
#What's up in each set

x = df.values[:, :-2]
y = df.values[:, -2]

x_test = df_test.values[:, :-2]
y_test = df_test.values[:, -2]

print('x train shape:', x.shape)
print('x test shape:', x_test.shape)
print('train class 0: {}, train class 1: {}'.format(len(y[y==0]), len(y[y==1])))
print('train class 0: {}, train class 1: {}'.format(len(y_test[y_test==0]), len(y_test[y_test==1])))

x train shape: (5246, 74)
x test shape: (1533, 74)
train class 0: 4936, train class 1: 310
train class 0: 1232, train class 1: 301


## Step 2: Model Choice

The first task is to decide which, of the large number of classifiers we have learned during this semester, would best suit our task and our data.

It would be possible to do brute force model comparison here - i.e. tune all models and compare which does best with respect to various benchmarks. However, it is also reasonable to do a first round of model comparison by running models (with out of the box parameter settings) on the training data and eliminating models which performed very poorly. 

In [15]:
def expected_score(model, x_test, y_test):
    overall = 0
    class_0 = 0
    class_1 = 0
    for i in range(100):
        sample = np.random.choice(len(x_test), len(x_test))
        x_sub_test = x_test[sample]
        y_sub_test = y_test[sample]
        
        overall += model.score(x_sub_test, y_sub_test)
        class_0 += model.score(x_sub_test[y_sub_test==0], y_sub_test[y_sub_test==0])
        class_1 += model.score(x_sub_test[y_sub_test==1], y_sub_test[y_sub_test==1])

    return pd.Series([overall / 100., 
                      class_0 / 100.,
                      class_1 / 100.],
                      index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])], 
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

In [16]:
#KNN
knn = KNN(n_neighbors=2)
knn.fit(x, y)

knn_scores = score(knn, x, y)
print('knn')

#Unweighted logistic regression
unweighted_logistic = LogisticRegression(C=1000)
unweighted_logistic.fit(x, y)

unweighted_log_scores = score(unweighted_logistic, x, y)
print('unweighted log')


#Weighted logistic regression
weighted_logistic = LogisticRegression(C=1000, class_weight='balanced')
weighted_logistic.fit(x, y)

weighted_log_scores = score(weighted_logistic, x, y)
print('weighted log')


#LDA
lda = LDA()
lda.fit(x, y)

lda_scores = score(lda, x, y)
print('lda')

#QDA
qda = QDA()
qda.fit(x, y)

qda_scores = score(qda, x, y)
print('qda')

#Decision Tree
tree = DecisionTree(max_depth=50, class_weight='balanced', criterion='entropy')
tree.fit(x, y)

tree_scores = score(tree, x, y)
print('tree')


#Random Forest
rf = RandomForest(class_weight='balanced')
rf.fit(x, y)

rf_scores = score(rf, x, y)

print('rf')

#SVC
svc = SVC(C=100, class_weight='balanced')
svc.fit(x, y)

svc_scores = score(svc, x, y)

print('svc')

knn
unweighted log
weighted log
lda


/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


qda
tree
rf
svc


In [17]:
#Score Dataframe
score_df = pd.DataFrame({'knn': knn_scores, 
                         'unweighted logistic': unweighted_log_scores,
                         'weighted logistic': weighted_log_scores,
                         'lda': lda_scores,
                         'qda': qda_scores,
                         'tree': tree_scores,
                         'rf': rf_scores, 
                         'svc': svc_scores})
score_df

,knn,lda,qda,rf,svc,tree,unweighted logistic,weighted logistic
overall accuracy,0.945864,0.937476,0.858559,0.988944,1.0,1.0,0.941289,0.705681
accuracy on class 0,1.000000,0.985616,0.869327,0.999797,1.0,1.0,0.999392,0.707050
accuracy on class 1,0.083871,0.170968,0.687097,0.816129,1.0,1.0,0.016129,0.683871


It looks like we can rule out KNN, LDA and unweighted logistic. 

**What we do:** We are going to pick weighted logistic regression and just tune the regularization parameter to beat the test benchmarks.

**What's probably good to do:** QDA, random forest, tree, SVC and weighted logistic are beating our train benchmarks as is. We will tune them to beat the test benchmarks by picking the model and parameter set with the highest CV accuracy.

In [18]:
Cs = 10.**np.arange(-3, 4, 1)
scores = []
for C in Cs:
    print('C:', C)
    weighted_log_scores = np.array([0., 0., 0.])
    kf = KFold(len(x), n_folds=10, shuffle=True, random_state=10)
    for train_index, test_index in kf:
        x_validate_train, x_validate_test = x[train_index], x[test_index]
        y_validate_train, y_validate_test = y[train_index], y[test_index]

        weighted_logistic = LogisticRegression(C=C, class_weight='balanced')
        weighted_logistic.fit(x_validate_train, y_validate_train)

        weighted_log_scores += score(weighted_logistic, x_validate_test, y_validate_test).values

    scores.append(weighted_log_scores / 10.)

scores = pd.DataFrame(np.array(scores).T, columns=[str(C) for C in Cs], index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

C: 0.001
C: 0.01
C: 0.1
C: 1.0
C: 10.0
C: 100.0
C: 1000.0


In [19]:
scores

,0.001,0.01,0.1,1.0,10.0,100.0,1000.0
overall accuracy,0.716161,0.701290,0.695951,0.693665,0.694046,0.692710,0.692139
accuracy on class 0,0.725485,0.709894,0.703826,0.701622,0.701813,0.700418,0.700397
accuracy on class 1,0.571205,0.569065,0.571008,0.567376,0.568633,0.568499,0.560197


To beat the future benchmark, we'll select the parameter which yields the highest accuracy on class 1 (while still beating the benchmark on class 0).

Now let's test our model on the test data:

In [20]:
#Weighted logistic regression
weighted_logistic = LogisticRegression(C=100, class_weight='balanced')
weighted_logistic.fit(x, y)
weighted_log_scores = score(weighted_logistic, x_test, y_test)
weighted_log_scores

overall accuracy       0.677756
accuracy on class 0    0.691558
accuracy on class 1    0.621262
dtype: float64

Yay, we beat all the benchmarks!

# Part II: Diagnosing Strains of the Semian Flu

From a public health perspective, we want to balance the cost of vaccinations, early interventions and the cost of treating flu complications of unvaccinated people. 

There are two different strains of the flu: strain 1 has a cheaper early intervention as well as a cheaper treatment for flu complications, but patients with strain 1 has a higher rate of developing complications if treated with the wrong intervention. Strain 2 has a more expensive early intervention as well as a more costly treatment for flu complications, but patients with strain 2 has a lower rate of developing complications if treated with the wrong intervention. With no intervention, flu patients develop complications at the same rate regardless of the strain. 

**Your task:** build a model to predict if a given patient has the flu and identify the flu strain. The state government of MA will use your model to inform public health policies: we will vaccinate people you've identified as healthy and apply corresponding interventions to patients with different strains of the flu. We have provided you with a function to compute the total expected cost of this policy decision that takes into account the cost of the vaccine, the interventions and the cost of the treatments for flu complications resulting from misdiagnosing patients. Your goal is to make sure your model produces a public health policy with the lowest associated expected cost.

**The deliverable:** a function called `flu_predict` which satisfies:

- input: `x_test`, a set of medical predictors for a group of patients
- output: `y_pred`, a set of labels, one for each patient; 1 for healthy, 2 for infected with strain 1, and 3 for infected with strain 2.

The MA state government will use your model to diagnose sets of future patients (held by us). You can expect that there will be an increase in the number of flu patients in any groups of patients in the future.

We provide you with some benchmarks for comparison.

**Three Baseline Models:** 
- expected cost on observed data: \$6,818,206.0, \$7,035,735.0, \$8,297,197.5
- time to build: 1 min

**Reasonable Model:** 
- expected cost on observed data: $6,300,000
- time to build: 20 min

**Grading:**
Your grade will be based on:
1. your model's ability to out-perform our benchmarks
2. your ability to carefully and thoroughly follow the data science pipeline (see lecture slides for definition)
3. the extend to which all choices are reasonable and defensible by methods you have learned in this class

In [21]:
#--------  cost
# A function that computes the expected cost of the public healthy policy based on the 
# classifications generated by your model
# Input: 
#      y_true (true class labels: 0, 1, 2)
#      y_pred (predicted class labels: 0, 1, 2)
# Returns: 
#      total_cost (expected total cost)

def cost(y_true, y_pred):
    cost_of_treatment_1 = 29500
    cost_of_treatment_2 = 45000
    cost_of_intervention_1 = 4150
    cost_of_intervention_2 = 4250
    cost_of_vaccine = 15
    
    prob_complications_untreated = 0.65
    prob_complications_1 = 0.30
    prob_complications_2 = 0.15
    
    trials = 1000    
    
    intervention_cost = cost_of_intervention_1 * len(y_pred[y_pred==1]) + cost_of_intervention_2 * len(y_pred[y_pred==2])

    vaccine_cost = cost_of_vaccine * len(y_pred[y_pred==0])
    
    false_neg_1 = ((y_true == 1) & (y_pred == 2)).sum()
    false_neg_2 = ((y_true == 2) & (y_pred == 1)).sum()
    
    untreated_1 = ((y_true == 1) & (y_pred == 0)).sum()    
    untreated_2 = ((y_true == 2) & (y_pred == 0)).sum()
    
    false_neg_1_cost = np.random.binomial(1, prob_complications_1, (false_neg_1, trials)) * cost_of_treatment_1
    false_neg_2_cost = np.random.binomial(1, prob_complications_2, (false_neg_2, trials)) * cost_of_treatment_2
    untreated_1_cost = np.random.binomial(1, prob_complications_untreated, (untreated_1, trials)) * cost_of_treatment_1
    untreated_2_cost = np.random.binomial(1, prob_complications_untreated, (untreated_2, trials)) * cost_of_treatment_2
    
    false_neg_1_cost = false_neg_1_cost.sum(axis=0)
    expected_false_neg_1_cost = false_neg_1_cost.mean()
    
    false_neg_2_cost = false_neg_2_cost.sum(axis=0)
    expected_false_neg_2_cost = false_neg_2_cost.mean()
    
    untreated_1_cost = untreated_1_cost.sum(axis=0)
    expected_untreated_1_cost = untreated_1_cost.mean()
    
    untreated_2_cost = untreated_2_cost.sum(axis=0)
    expected_untreated_2_cost = untreated_2_cost.mean()
    
    total_cost = vaccine_cost + intervention_cost + expected_false_neg_1_cost + expected_false_neg_2_cost + expected_untreated_1_cost + expected_untreated_2_cost
    
    return total_cost

We're just going to take the weighted logistic model, again, and tune the regularization parameter to both beat the benchmark on the observed data and minimize expected cost on unseen data (i.e. prevent ***overfitting***). Instead of using 'balanced' class weights, we're using a custom weighting scheme for the three classes (this parameter should really be tuned!).

It would probally also be go through the whole "choosing a model, tuning these models"-process again, this time to minimize cost.

**Note:** Be aware that the cost is now sensitive to sample size! The smaller the pool of patients the less the cost. If you are evaluating cost on a held-out test set then you can artificially make the cost very small. The benchmarks we give are for the entire training set.

In [22]:
x = df.values[:, :-2]
y = df.values[:, -1]
y = y - 1

x_test = df_test.values[:, :-2]
y_test = df_test.values[:, -1]

y_test = y_test - 1

In [23]:
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1]), 
                                                 model.score(x_test[y_test==2], y_test[y_test==2]), 
                                                 cost(y_test, model.predict(x_test))],
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1', 'accuracy on class 2', 'total cost'])

In [25]:
Cs = 10.**np.arange(-3, 4, 1)
scores = []
for C in Cs:
    print('C:', C)
    weighted_log_scores = np.array([0., 0., 0., 0., 0.])
    kf = KFold(len(x), n_folds=10, shuffle=True, random_state=10)
    for train_index, test_index in kf:
        x_validate_train, x_validate_test = x[train_index], x[test_index]
        y_validate_train, y_validate_test = y[train_index], y[test_index]

        weighted_logistic = LogisticRegression(C=C, class_weight={0:0.7, 1:10, 2:10})
        weighted_logistic.fit(x_validate_train, y_validate_train)

        weighted_log_scores += score(weighted_logistic, x_validate_test, y_validate_test).values

    scores.append(weighted_log_scores / 10.)

scores = pd.DataFrame(np.array(scores).T, columns=[str(C) for C in Cs], index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1', 'accuracy on class 2', 'total cost'])

C: 0.001
C: 0.01
C: 0.1
C: 1.0
C: 10.0
C: 100.0
C: 1000.0


In [70]:
scores

,0.001,0.01,0.1,1.0,10.0,100.0,1000.0
overall accuracy,0.930429,0.928141,0.929476,0.928904,0.928714,0.929096,0.929095
accuracy on class 0,0.984596,0.981556,0.982162,0.982772,0.982165,0.981959,0.981950
accuracy on class 1,0.069623,0.076190,0.084405,0.069524,0.070119,0.080952,0.082894
accuracy on class 2,0.074134,0.105087,0.139177,0.083225,0.108983,0.133983,0.124892
total cost,658563.900000,658571.850000,652914.050000,658370.650000,654790.300000,651038.850000,654992.250000


In [76]:
#Weighted logistic regression
weighted_logistic = LogisticRegression(C=100, class_weight={0:0.7, 1:10, 2:10})
weighted_logistic.fit(x, y)
weighted_log_scores = score(weighted_logistic, x, y)
weighted_log_scores

overall accuracy             0.933473
accuracy on class 0          0.985008
accuracy on class 1          0.092511
accuracy on class 2          0.168675
total cost             6286907.000000
dtype: float64

In [77]:
#Weighted logistic regression
weighted_log_scores = score(weighted_logistic, x_test, y_test)
weighted_log_scores

overall accuracy             0.802348
accuracy on class 0          0.985390
accuracy on class 1          0.064677
accuracy on class 2          0.030000
total cost             6341513.500000
dtype: float64

In [26]:
print('minimimum cost on train:', cost(y, y))
print('minimimum cost on test:', cost(y_test, y_test))

minimimum cost on train: 1368840.0
minimimum cost on test: 1277630.0


In [27]:
print('simple model cost on train:', cost(y, np.array([0] * len(y))))
print('simple model cost on test:', cost(y_test, np.array([0] * len(y_test))))

simple model cost on train: 6859750.5
simple model cost on test: 6812989.0


In [28]:
print('simple model cost on train:', cost(y, np.array([1] * len(y))))
print('simple model cost on test:', cost(y_test, np.array([1] * len(y_test))))

simple model cost on train: 22326200.0
simple model cost on test: 7037445.0


In [29]:
print('simple model cost on train:', cost(y, np.array([2] * len(y))))
print('simple model cost on test:', cost(y_test, np.array([2] * len(y_test))))

simple model cost on train: 24297163.5
simple model cost on test: 8294454.0


Yay! We beat the benchmarks on the observed data and did pretty good on test data!

# Autograding function

In [33]:
accuracy = lambda y_true, y_pred: (((y_true - y_pred) == 0).sum() * 1.) / len(y_true)

# function to check - did they beat our benchmarks?
# You can either pass it the flu_predict function the student wrote or the
# file name of the y-labels.
#
# predict: function of type (array -> array)
# pred_y_file_name: file name of where their predicted y-labels live
# data_preprocessing: if false x-train data will be fed to flu_predict with no processing if true data will be encoded with 0 filled in for NaN
# cost: if true computes the expect cost
# 
# return some string indicating result of comparison with benchmark

def beat_benchmark(flu_predict=None, pred_y_file_name=None, data_preprocessing=False):
    
    acc_bm_0 = .69
    acc_bm_1 = .60
    
    acc_rm_0 = .5
    acc_rm_1 = .5
    
    df_test = pd.read_csv('data/flu_test.csv')
    df_test = df_test[~np.isnan(df_test['flu'])]
    df_test['flutype'] = df_test['flutype'] - 1
    
    y_true = df_test.values[:, -2]
    
    if flu_predict is not None:
        if data_preprocessing:                       
            encode = preprocessing.LabelEncoder()
            for column in df_test.columns:
                if df_test[column].dtype == np.object:
                    df_test.loc[:, column] = encode.fit_transform(df_test[column])

            df_test = df_test.fillna(0)
            
        x = df_test.values[:, :-2]
        
        y_pred = flu_predict(x)
        
    elif pred_y_file_name is not None:
        df_y_pred = pd.read_csv(pred_y_file_name)
        y_pred = df_y_pred.values[:, -1]
        
    else:
        return 'params ill-specified'
    
    acc_0 = accuracy(y_true[y_true == 0], y_pred[y_true == 0])
    acc_1 = accuracy(y_true[y_true == 1], y_pred[y_true == 1])
    
    if acc_0 > acc_rm_0 and acc_1 > acc_rm_1:
        print ('accuracies: {}, {}'.format(acc_0, acc_1))
        return "accuracy: beats all benchmarks :)"
    elif acc_0 > acc_bm_0 and acc_1 > acc_bm_1:
        print ('accuracies: {}, {}'.format(acc_0, acc_1))
        return "accuracy: beats only baseline models :/"
    else:
        print ('accuracies: {}, {}'.format(acc_0, acc_1))
        return "accuracy: beats no benchmarks :("

In [34]:
# Example 0: using the autograding function

def flu_predict(x):
    return np.array([0] * len(x))

print(beat_benchmark(flu_predict, data_preprocessing=False))

accuracies: 1.0, 0.0
accuracy: beats no benchmarks :(


In [35]:
# Example 1: using the autograding function
print(beat_benchmark(pred_y_file_name='data/Example_1.csv'))

accuracies: 0.7191558441558441, 0.6112956810631229
accuracy: beats all benchmarks :)
